#### Folium Map
The following is an interactive map which shows the distance between each substation and outage. The code provides an html file which can later be served on our website. 

In [ ]:
import folium
import sys
import pandas as pd

# Allow importing from parent directory by temporarily moving the CWD up one level
# Very hacky, but there literally isn't a simpler way (in Jupyter)
sys.path.append("..")
from common import get_dataframe_from_pipeline

outages = get_dataframe_from_pipeline("../pipeline/3.csv.gz")
# Drop the path back down after import
sys.path.pop()

substations = pd.read_csv("../_datasets/BCSubstationLocations.csv")
populatedPlaces = pd.read_csv("../_datasets/BCPopulatedPlaces.csv")

In [ ]:
# create an open street map of the lower mainland
map = folium.Map(location=(49.1934, -122.89), zoom_start=10)


Resources :

[Tutorial (YT)](https://www.youtube.com/watch?v=X8CoVymMOLw)

[Making the markings](https://python-visualization.github.io/folium/latest/getting_started.html)

[Making the markings + maps (YT)](https://www.youtube.com/watch?v=t9Ed5QyO7qY)

[PolyLine](https://python-visualization.github.io/folium/latest/user_guide.html)

You can find the valid map icons [here](https://getbootstrap.com/docs/3.3/components/).

Icons colour has to be : {'green', 'darkred', 'red', 'lightblue', 'darkgreen', 'white', 'lightgreen', 'orange', 'purple', 'lightred', 'black', 'beige', 'blue', 'cadetblue', 'gray', 'lightgray', 'pink', 'darkpurple', 'darkblue'}

In [ ]:
# TODO change the size of the popup, and 
# change what info it displays (check https://stackoverflow.com/questions/63152298/updating-folium-changed-the-popup-box-width)
# create a layer for populated places, substations, outages, and lines
populatedPlaceLayer = folium.FeatureGroup(name="Populated Places")
substationsLayer = folium.FeatureGroup(name="Substations")
outagesLayer = folium.FeatureGroup(name="Outages")
subLinesLayer = folium.FeatureGroup(name="Substation and Outages Distance")
popLinesLayer = folium.FeatureGroup(name="Populate Places and Outages Distance")

# markers for substations
for (index, row) in substations.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = f"This is the '{row['name']}' substation.",
        tooltip = 'substation',
        icon = folium.Icon(color='blue', icon='tint')
    ).add_to(substationsLayer) # add to the substations layer

# markers for populated places
for (index, row) in populatedPlaces.iterrows():
    folium.Marker(
        location = [row['latitude'], row['longitude']],
        popup = f"This populated place is located in {row['geographical name']}.",
        tooltip = 'populated place',
        icon = folium.Icon(color='cadetblue', icon='home')
    ).add_to(populatedPlaceLayer) # add to the populated places layer

# markers for outages
for (index, row) in outages.iterrows():
    folium.Marker(
        location = [row['outageLatitude'], row['outageLongitude']],
        popup = f"The outage id '{row['outageId']}' occurred at {row['dateOn']}. It is tagged with the cause: {row['outageCause']}",
        tooltip = 'outage',
        icon = folium.Icon(color='red', icon='flash')
    ).add_to(outagesLayer) # add to the outages layer

    outageCoordinates = [row['outageLatitude'], row['outageLongitude']]
    nearestSubstation = substations.set_index("id").loc[row['nearestSubstationId']]
    substationCoordinates = [nearestSubstation['latitude'], nearestSubstation['longitude']]

    # substation and outages
    folium.PolyLine(
        locations = [outageCoordinates, substationCoordinates], 
        color = "#6a1ca6", 
        popup = f"This line connects '{nearestSubstation['name']}' substation and outage id '{row['outageId']}'. It is {round(row['outageToSubstationDistance'], 2)} km long.",
        weight = 3, 
        opacity = 1
    ).add_to(subLinesLayer)

    # TODO draw lines for populated place to outage
    nearestPop = populatedPlaces.set_index("id").loc[row['nearestPopulatedPlaceId']]
    popCoordinates = [nearestPop['latitude'], nearestPop['longitude']]

    # substation and outages
    folium.PolyLine(
        locations = [outageCoordinates, popCoordinates], 
        color = "#5D3FD3", 
        popup = f"This line connects stuff",
        weight = 3, 
        opacity = 1
    ).add_to(popLinesLayer)

# add layers to map
populatedPlaceLayer.add_to(map)
substationsLayer.add_to(map)
outagesLayer.add_to(map)
subLinesLayer.add_to(map)
popLinesLayer.add_to(map)

# LayerControl allows toggling between layers
folium.LayerControl().add_to(map)

map

In [ ]:
## Ignore for now, still need will delete later

# # make markers for outages and substations
# for (index, row) in substations.iterrows():
#   # marker for substation
#   folium.Marker(
#       location=[row['latitude'], row['longitude']],
#       popup= f"This is the '{row['name']}' substation.",
#       tooltip='substation',
#       icon=folium.Icon(color='blue', icon='tint')
#   ).add_to(map)

# for (index, row) in outages.iterrows():
#     # marker for outages
#     # TODO change the size of the popup, and 
#     # change what info it displays (check https://stackoverflow.com/questions/63152298/updating-folium-changed-the-popup-box-width)
#     folium.Marker(
#         location=[row['latitude'], row['longitude']],
#         popup= f"The outage id '{row['outageId']}' occured at {row['dateOn']}. It is tagged with the cause : {row['outageCause']}",
#         tooltip='outage',
#         icon=folium.Icon(color='red', icon='flash')
#     ).add_to(map)

#     outageCoordinates = [row['latitude'], row['longitude']]

#     nearestSubstation = substations.set_index("id").loc[row['nearestSubstationId']]
#     substationCoordinates = [nearestSubstation['latitude'], nearestSubstation['longitude']]

#     # connect outage to nearest substation
#     folium.PolyLine(
#         locations=[outageCoordinates, substationCoordinates], 
#         color="#6a1ca6", 
#         popup= f"This line connects '{nearestSubstation['name']}' substation and outage id '{row['outageId']}'. It is {round(row['outageToSubstationDistance'], 2)}km long.",
#         weight=3, 
#         opacity=1
#     ).add_to(map)

# map


# # make markers for outages and substations
# for (index, row) in outages.iterrows():
#     # marker for outages
#     # TODO change the size of the popup, and 
#     # change what info it displays (check https://stackoverflow.com/questions/63152298/updating-folium-changed-the-popup-box-width)
#     folium.Marker(
#         location=[row['outageLatitude'], row['outageLongitude']],
#         popup= f"The outage id '{row['outageId']}' occured at {row['dateOn']}. It is tagged with the cause : {row['outageCause']}",
#         tooltip='outage',
#         icon=folium.Icon(color='red', icon='flash')
#     ).add_to(map)
    
#     # marker for substation
#     folium.Marker(
#         location=[row['substationLatitude'], row['substationLongitude']],
#         popup= f"This is the '{row['substationName']}' substation. It covers part of the {row['outageMunicipality']} municipality.",
#         tooltip='substation',
#         icon=folium.Icon(color='blue', icon='tint')
#     ).add_to(map)

#     outageCoordinates = [row['outageLatitude'], row['outageLongitude']]
#     substationCoordinates = [row['substationLatitude'], row['substationLongitude']]

#     # connect outage to nearest substation
#     folium.PolyLine(
#         locations=[outageCoordinates, substationCoordinates], 
#         color="#6a1ca6", 
#         popup= f"This line connects '{row['substationName']}' substation and outage id '{row['outageId']}'. It is {round(row['outageToSubstationDistance'], 2)}km long.",
#         weight=3, 
#         opacity=1
#     ).add_to(map)

# map

In [ ]:
import os
# Save the map
# Make directory if it doesn't exist
if not os.path.exists("html"):
  os.makedirs("html")

map.save('html/mapOfDistance.html')

In [ ]:
# opens the map in your browser
# !open mapOfDistance.html